In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/clase_09/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Limpieza de datos. Apply. Expresiones regulares.

La limpieza es un paso necesario en todo proyecto de datos. 

Podemos resumir el proceso de limpieza de datos en las siguientes cinco tareas:

**1. Resolver problemas de formato y asignar tipos de datos correctos.**

Por ejemplo, cuando al pasar de CSV a Pandas una fecha no se importa correctamente como puede ser el caso de un campo fecha donde se importa 20090609231247 en lugar de 2009-06-09 23:12:47.

El formato en que se encuentran los datos determina qué operaciones pueden realizarse sobre ellos.

**2. Estandarizar categorías.**

Cuando los datos se recolectaron con un sisstema que no tiene valores tipificadaos, valores que representan la misma categoría pueden estar expresados de forma distinta. Por ejemplo: Arg, AR, Argentina

**3. Corregir valores erróneos.**

Por ejemplo: un valor numérico o inválido para describir el género; o una edad representada por un número negativo o mucho mayor que 100.

**4. Completar datos faltantes.**

Los datasets del mundo real suelen venir con datos faltantes que responden a información que se perdió o nunca se recolectó. Existen varias técnicas para completar datos faltantes. Al proceso de completar datos faltantes se lo llama "imputación".

**5. Organizar el dataset.**

Es importante estructurar las filas y columnas de la forma más conveniente. Para hacerlo se pueden aplicar las reglas del "tidy data".



## Dataset

El Met (Metropolitan Museum of Art) provee datasets de información de más de 420.000 piezas de arte en su colección.

https://github.com/metmuseum/openaccess/

Los problemas que presentan este dataset incluyen:
* Valores faltantes
* Posibles duplicaciones
* Campos con mezcla de campos de tipo numérico con cadenas de caracteres

En esta clase vamos a detectar algunos de los qué campos presentan problemas, y solucionarlos usando las herramientas que adquirimos en las prácticas guiadas.


## Ejercicio 1:

Vamos a leer en la variable `data` los datos del archivo /M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv en un `DataFrame` de pandas con el método `read_csv` 

Veamos cuántas filas y columnas tiene el DataFrame data

In [2]:
import pandas as pd
import numpy as np
import re 
%matplotlib inline

In [3]:
data = pd.read_csv('../Data/MetObjects_sample.csv', sep=',')

In [4]:
data.shape
print('El dataframe Data tiene {} filas y {} columnas '.format(data.shape[0],data.shape[1]))

El dataframe Data tiene 4743 filas y 52 columnas 


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4743 entries, 0 to 4742
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               4743 non-null   int64  
 1   Object Number            4743 non-null   object 
 2   Is Highlight             4743 non-null   bool   
 3   Is Public Domain         4743 non-null   bool   
 4   Is Timeline Work         4743 non-null   bool   
 5   Object ID                4743 non-null   int64  
 6   Department               4743 non-null   object 
 7   AccessionYear            3743 non-null   object 
 8   Object Name              4655 non-null   object 
 9   Title                    3914 non-null   object 
 10  Culture                  2318 non-null   object 
 11  Period                   1319 non-null   object 
 12  Dynasty                  184 non-null    object 
 13  Reign                    71 non-null     object 
 14  Portfolio               

## Ejercicio 2: Formato y tipos de datos

Vamos a ver de qué tipo de datos es cada columna del DataFrame, y vamos a convertir o dar formato **a alguna de las columnas** que tienen tipo de datos incorrecto.

### 2.a Detectar las columnas que tienen tipo de datos incorrecto

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4743 entries, 0 to 4742
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               4743 non-null   int64  
 1   Object Number            4743 non-null   object 
 2   Is Highlight             4743 non-null   bool   
 3   Is Public Domain         4743 non-null   bool   
 4   Is Timeline Work         4743 non-null   bool   
 5   Object ID                4743 non-null   int64  
 6   Department               4743 non-null   object 
 7   AccessionYear            3743 non-null   object 
 8   Object Name              4655 non-null   object 
 9   Title                    3914 non-null   object 
 10  Culture                  2318 non-null   object 
 11  Period                   1319 non-null   object 
 12  Dynasty                  184 non-null    object 
 13  Reign                    71 non-null     object 
 14  Portfolio               

### 2.b AccessionYear

Analizar la columna AccessionYear que fue leída como object, y debería ser int.

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos extraer el dato año de los valores no numéricos, y crear una nueva columna en el DataFrame de tipo int que se llame AccessionYearClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

Observación: si la columna tiene valores NaN no vamos a poder convertirla al tipo int. Una opción es reemplazar los valores NaN por algún entero antes de convertir (`fillna`). La otra opción es dejar la columna como tipo float (el tipo de NaN es float).


In [7]:
data["AccessionYear"].unique()

array([nan, '2005-02-15', '2020-03-23', '1986-12-29', '1992', '1991',
       '1975', '1930', '1874', '1915', '1920', '1921', '1916', '1929',
       '1907', '1989', '1962', '1917', '1895', '1950', '1896', '1924',
       '1951', '1970', '1958', '1938', '2006', '1953', '1891', '1965',
       '1966', '2009', '1911', '1947', '1925', '1955', '1914', '1928',
       '2012', '1926', '2015', '1879', '1887', '1949', '1902', '1981',
       '1979', '1899', '1918', '1906', '1912', '2001', '1985', '1976',
       '1946', '1927', '1973', '1941', '1900', '2019', '2013', '1942',
       '1968', '1967', '1988', '1986', '1933', '1913', '1909', '1977',
       '1922', '1964', '2018', '1936', '1948', '1919', '1954', '1959',
       '1931', '1943', '1908', '1945', '1932', '1982', '1939', '1881',
       '1956', '2002', '1910', '2008', '1923', '1961', '1937', '2004',
       '1971', '1905', '1893', '1898', '1994', '1960', '1957', '1880',
       '1984', '2016', '1944', '1969', '1972', '1871', '1987', '2007',
       

In [8]:
# hay valores nulos
print("En total en la columna AccessionYear hay {} NaN Values".format(data.AccessionYear.isnull().sum()))

En total en la columna AccessionYear hay 1000 NaN Values


In [9]:
#Se distribuyen de la siguiente manera
data.AccessionYear.value_counts()

1963          220
1917          140
1874          126
1929           99
2011           98
             ... 
1986-12-29      1
1871            1
2005-02-15      1
1873            1
1892            1
Name: AccessionYear, Length: 142, dtype: int64

In [10]:
patron = re.compile("\d{4}")
patron_data = data['AccessionYear'].apply(lambda x: x if x is np.NaN else patron.search(str(x)))
#Ahora voy a crear mi mascara booleana
mascara = patron_data.notnull()

data.loc[mascara, 'AccessionYearClean'] = patron_data[mascara].apply(lambda x: x.group(0))

In [11]:
data.loc[mascara, ['AccessionYear','AccessionYearClean']]

,AccessionYear,AccessionYearClean
1000,2005-02-15,2005
1001,2020-03-23,2020
1002,1986-12-29,1986
1003,1992,1992
1004,1991,1991
...,...,...
4738,1941,1941
4739,1979,1979
4740,1979,1979
4741,1953,1953


In [12]:
(data.loc[data.AccessionYearClean.notnull(),'AccessionYearClean']).astype(int)

1000    2005
1001    2020
1002    1986
1003    1992
1004    1991
        ... 
4738    1941
4739    1979
4740    1979
4741    1953
4742    1963
Name: AccessionYearClean, Length: 3743, dtype: int64

In [13]:
data.AccessionYearClean.unique()
#Aca podemos ver que nos falta trabajar con los NaN

array([nan, '2005', '2020', '1986', '1992', '1991', '1975', '1930',
       '1874', '1915', '1920', '1921', '1916', '1929', '1907', '1989',
       '1962', '1917', '1895', '1950', '1896', '1924', '1951', '1970',
       '1958', '1938', '2006', '1953', '1891', '1965', '1966', '2009',
       '1911', '1947', '1925', '1955', '1914', '1928', '2012', '1926',
       '2015', '1879', '1887', '1949', '1902', '1981', '1979', '1899',
       '1918', '1906', '1912', '2001', '1985', '1976', '1946', '1927',
       '1973', '1941', '1900', '2019', '2013', '1942', '1968', '1967',
       '1988', '1933', '1913', '1909', '1977', '1922', '1964', '2018',
       '1936', '1948', '1919', '1954', '1959', '1931', '1943', '1908',
       '1945', '1932', '1982', '1939', '1881', '1956', '2002', '1910',
       '2008', '1923', '1961', '1937', '2004', '1971', '1905', '1893',
       '1898', '1994', '1960', '1957', '1880', '1984', '2016', '1944',
       '1969', '1972', '1871', '1987', '2007', '1980', '2000', '1990',
       '1

In [14]:
(data.loc[data.AccessionYearClean.isnull(), 'AccessionYearClean']).astype(float)
#Convertimos a float los NaN

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
995   NaN
996   NaN
997   NaN
998   NaN
999   NaN
Name: AccessionYearClean, Length: 1000, dtype: float64

## Ejercicio 3: Categorias - Valores erróneos

Miremos ahora el campo "Artist Gender"

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos definir como categorías válidas Male, Female y Unknown

Y crear una nueva columna en el DataFrame que se llame ArtistGenderClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

¿Podemos deducir cómo está representada la categoría Male en el dataset original?

Nota: La propuesta que hacemos para limpiar este campo no es del todo correcta, y vamos a ver por qué más adelante. Pero sirve como ejercicio.


In [15]:
data['Artist Gender'].unique()

array([nan, '|', 'Female|', '||', 'Female', '|Female', '|||', '||Female',
       '|||||', '|Female||', '|||||||', '||||', '||||||', '||Female|',
       'Female||', '|Female|', 'Female||Female', '|||||||||||||||||',
       '|||||||||||', '|||||||||||||||Female|||',
       '|Female|Female|||Female', '|||Female|||||', '||||||||',
       'Female|Female'], dtype=object)

In [16]:
data['Artist Gender'].value_counts()

|                           583
||                          170
Female                       81
Female|                      43
|||                          41
|Female                      23
||||                         20
||||||                        7
|||||||                       3
|||||                         3
Female||                      3
|Female|                      2
|||||||||||                   2
Female||Female                2
||Female|                     1
||Female                      1
|||Female|||||                1
|||||||||||||||Female|||      1
|Female|Female|||Female       1
|Female||                     1
|||||||||||||||||             1
Female|Female                 1
||||||||                      1
Name: Artist Gender, dtype: int64

In [17]:
data['Artist Gender'].isnull().sum()

3751

In [18]:
# cada | representa un hombre
# Si solo quieren definir como validas las categorias Female(todas mujeres), Male(todos hombres), unknow(las demas)

In [19]:
patron_gender = re.compile('^\|$')

male = data["Artist Gender"].apply(lambda x: x if x is np.NaN else re.sub(patron_gender,"Male",str(x)))

mask_male = male.notnull()

data.loc[mask_male,"Artist Gender Clean"] = male[mask_male]

In [20]:
data["Artist Gender Clean"].unique()

array([nan, 'Male', 'Female|', '||', 'Female', '|Female', '|||',
       '||Female', '|||||', '|Female||', '|||||||', '||||', '||||||',
       '||Female|', 'Female||', '|Female|', 'Female||Female',
       '|||||||||||||||||', '|||||||||||', '|||||||||||||||Female|||',
       '|Female|Female|||Female', '|||Female|||||', '||||||||',
       'Female|Female'], dtype=object)

In [21]:
patron_gender2 = re.compile('(^Female$)')

female = data["Artist Gender"].apply(lambda x: x if x is np.NaN  else patron_gender2.match(str(x)))

mask_female = female.notnull()

In [22]:
patron_gender3 = re.compile('(\w)?\|+')

patron_gender33 = data["Artist Gender Clean"].apply(lambda x: x if x is np.NaN else patron_gender3.search(str(x)))

patron_gender3_bool = patron_gender33.notnull()

mask_unknow = np.logical_or((~np.logical_or(mask_female, mask_male)),patron_gender3_bool)
data.loc[mask_unknow, "Artist Gender Clean"] = "Unknow"

data["Artist Gender Clean"].value_counts()

Unknow    4079
Male       583
Female      81
Name: Artist Gender Clean, dtype: int64

## Ejercicio 4: Imputación

Vamos a analizar ahora los campos "Object Date", "Object Begin Date", "Object End Date"

**4.a ¿Cuántos valores nulos hay en "Object Date"? ¿Cuántos en "Object Begin Date"? ¿Cuántos en "Object End Date"?**


In [23]:
data["Object Date"].isnull().sum()

1570

In [24]:
data["Object Begin Date"].isnull().sum()

0

In [25]:
data["Object End Date"].isnull().sum()

0

**4.b Usaremos los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

1) Vamos a crear una columna nueva ("Object Date 4b") donde copiamos todos los datos de Object Date (para no cambiar los valores originales y nos sirvan para el próximo ejercicio)

2) Vamos a rellenar la columna "Object Date 4b" con la estrategia que planteamos.

In [26]:
data["Object Date 4b"]=data["Object Date"]

**4.c Usaremos aleatoriamente los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

Para eso definimos una función get_fill_value que recibe como parámetro una fila da data, y si el valor del campo "Object Date" es nulo devuelve aleatoriamente el valor del campo "Object Begin Date" u "Object End Date" de ese registro.

(En este ejercicio practicamos todo!)

In [27]:
def get_fill_value(x):
    rng = np.random.default_rng()
    col = rng.integers(low=0,high=101,size=1)
    if col%2==0:
        return x["Object Begin Date"]
    else:
        return x["Object End Date"]
    
object_bool=data["Object Date 4b"].isnull()

data.loc[object_bool,["Object Date 4b"]] = data.apply(lambda x : get_fill_value(x), axis=1)

In [28]:
data.loc[object_bool,["Object Date","Object Date 4b","Object Begin Date","Object End Date"]]

,Object Date,Object Date 4b,Object Begin Date,Object End Date
51,NaN,0,0,0
103,NaN,330,-7000,330
122,NaN,0,0,0
136,NaN,0,0,0
159,NaN,0,0,0
...,...,...,...,...
2498,NaN,1440,1440,1450
2499,NaN,-7000,-7000,-30
2500,NaN,1644,1644,1911
2501,NaN,1536,1526,1536


Vamos a contar ahora cuántos registros rellenó con los valores de Object Begin Date y cuántos con Object End Date

In [29]:
(data["Object Begin Date"] == data['Object Date 4b']).sum()

867

In [30]:
(data["Object End Date"] == data['Object Date 4b']).sum()

860

Nota, la suma de ambos registros no da la cantidad total de filas, ya que los valores 0 donde ambas columnas son iguales, tambien los esta contando.

## Nota: Organizar el dataset

Para llevar a cabo esta tarea necesitamos algunas herramientas que veremos en la segunda parte de Pandas.

En esa clase vamos a volver a este ejercicio y resolver este punto.

Los campos que vamos a analizar son "Artist Nationality" y "Artist Display Name"

In [31]:
data["Artist Nationality"].value_counts()

American                                                   329
Austrian                                                   153
French                                                     146
Japanese                                                    96
Italian                                                     93
                                                          ... 
American, born Austria                                       1
French, born Germany|Italian                                 1
Netherlandish|Italian|Italian                                1
Italian|Italian|Italian|Italian|Italian|Italian|Italian      1
British, born Cuba                                           1
Name: Artist Nationality, Length: 167, dtype: int64

In [32]:
data["Artist Display Name"].value_counts() 

Walker Evans                                                                                                       35
Unknown                                                                                                            32
W. Duke, Sons & Co.                                                                                                21
Kinney Brothers Tobacco Company                                                                                    20
Unidentified Artist                                                                                                19
                                                                                                                   ..
Lucas Cranach the Elder|Albrecht Dürer                                                                              1
Burton Silverman                                                                                                    1
S. Arlent Edwards                                       